#6.8 Stuff LCEL Chain

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langsmith import traceable
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough


In [5]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.1)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = TextLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

@traceable
def generate_response(chain, prompt):
    return chain.invoke(prompt)


In [6]:
retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

generate_response(chain, "Describe Victory Mansions")

Failed to use model_dump to serialize <class 'langchain_core.runnables.base.RunnableSequence'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_community.vectorstores.faiss.FAISS'>)


AIMessage(content="Victory Mansions is a building with a hallway that smells of boiled cabbage and old rag mats.  A large poster of a man's face is tacked to the wall in the hallway.  The building has seven flights of stairs; the lift is seldom working.  From the roof, you can see the four Ministries simultaneously.\n", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-69350249-8c0c-48a6-80ce-a87f08201b39-0', usage_metadata={'input_tokens': 2040, 'output_tokens': 68, 'total_tokens': 2108, 'input_token_details': {'cache_read': 0}})